In [2]:
##INNOCENTKISOKA#


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-t/data/train_20_DLL_ass4.pkl
/kaggle/input/data-t/data/test_20_DLL_ass4.pkl
/kaggle/input/data-t/data/dummy_20_DLL_ass4 (1).pkl
/kaggle/input/data-t/data/valid_20_DLL_ass4.pkl


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pickle
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
from tqdm import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.amp import GradScaler

In [4]:
class TSPDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        G, opt_tour = self.data[idx]
        n = G.number_of_nodes()
        attr = nx.get_node_attributes(G, 'pos')
        X = []
        for i in range(n):
            X.append(torch.tensor(attr[i], dtype=torch.float32))

        return torch.stack(X), torch.tensor(opt_tour)


In [5]:
def create_dataloaders(train_path, val_path, test_path, batch_size=32):
    """
    Create DataLoader objects for training, validation, and testing.
    Args:
        train_path (str): Path to training data pickle file.
        val_path (str): Path to validation data pickle file.
        test_path (str): Path to test data pickle file.
        batch_size (int): Batch size for the dataloaders.
    Returns:
        tuple: (train_loader, val_loader, test_loader)
    """
    try:
        # Use only one argument for TSPDataset
        #train_dataset = TSPDataset('/kaggle/input/data-t/data/train_20_DLL_ass4.pkl')
        X = pickle.load(open(train_path, 'rb'))
        train_dataset = TSPDataset(X)
        X = pickle.load(open(val_path, 'rb'))
        val_dataset = TSPDataset(X)
        X = pickle.load(open(test_path, 'rb'))
        test_dataset = TSPDataset(X)
        #val_dataset = TSPDataset('/kaggle/input/data-t/data/valid_20_DLL_ass4.pkl')
        #test_dataset = TSPDataset('/kaggle/input/data-t/data/test_20_DLL_ass4.pkl')
        
        train_loader = DataLoader(
            train_dataset,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,  # Changed to 0 to avoid potential multiprocessing issues
            pin_memory=True
        )
        
        val_loader = DataLoader(
            val_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True
        )
        
        test_loader = DataLoader(
            test_dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=0,
            pin_memory=True
        )
        
        return train_loader, val_loader, test_loader
    
    except Exception as e:
        print(f"Error creating dataloaders: {str(e)}")
        raise


In [6]:
# Cell 3: Positional Encoding
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        
        self.register_buffer('pe', pe)
        
    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [7]:
# Cell 4: Transformer Model
class TSPTransformer(nn.Module):
    def __init__(self, d_model=128, nhead=8, num_encoder_layers=6, num_decoder_layers=6, n_nodes=20):
        super().__init__()
        self.d_model = d_model
        
        # Input embedding for node coordinates (2 -> d_model)
        self.input_embedding = nn.Linear(2, d_model)
        
        # Node embedding for decoder (n_nodes -> d_model)
        self.node_embedding = nn.Embedding(n_nodes, d_model)
        
        # Positional encoding for decoder
        self.pos_encoder = PositionalEncoding(d_model)
        
        # Transformer components
        self.transformer = nn.Transformer(
            d_model=d_model,
            nhead=nhead,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            batch_first=True
        )
        
        # Output layer
        self.out = nn.Linear(d_model, n_nodes)
        
    def create_mask(self, size):
        mask = torch.triu(torch.ones(size, size), diagonal=1).bool()
        return mask
    
    def forward(self, x, y):
        # Encoder
        enc_input = self.input_embedding(x)
        
        # Decoder
        dec_input = self.node_embedding(y)
        dec_input = self.pos_encoder(dec_input)
        
        # Create causal mask for decoder
        tgt_mask = self.create_mask(y.size(1)).to(x.device)
        
        # Run through transformer
        out = self.transformer(
            enc_input, 
            dec_input,
            tgt_mask=tgt_mask
        )
        
        return self.out(out)

In [8]:
# Cell 5: Evaluation Functions
from networkx.algorithms.approximation import greedy_tsp

def tour_length(G, tour):
    """
    Compute the length of a tour. A tour is a list having elements 0 and -1 equal
    """
    n = len(tour) - 1
    assert tour[0] == tour[-1], "Not valid tour"
    estimated = 0
    for i in range(n):
        estimated += G[tour[i]][tour[i + 1]]['weight']
    return estimated

def greedy_algorithm(G):
    """
    Run the value of the greedy approximation algorithm on graph G
    """
    return tour_length(G, greedy_tsp(G, weight='weight'))

def random_tour(G, seed=42):
    """
    Return the value of a random tour
    """
    np.random.seed(seed)
    n = G.number_of_nodes()
    tour = [0]
    for i in range(1, n):
        next_node = np.random.choice([j for j in range(n) if j not in tour])
        tour.append(next_node)
    tour.append(0)
    return tour_length(G, tour)

def transformer_tsp(G, model, DEVICE='cpu'):
    """
    Evaluate your (trained) model on G
    """
    model.eval()
    n = G.number_of_nodes()
    
    # Get node coordinates
    attr = nx.get_node_attributes(G, 'pos')
    x = []
    for i in range(n):
        x.append(torch.tensor(attr[i], dtype=torch.float32))

    # From list of tensors to tensor
    x = torch.stack(x)    # This may be not needed

    tour = [0]
    y = torch.tensor(tour, dtype=torch.long)
    x = x.to(DEVICE).unsqueeze(0)
    y = y.to(DEVICE).unsqueeze(0)
    
    with torch.no_grad():
        out = model(x, y)
        
        while len(tour) < n:
            _, idx = torch.topk(out, n, dim=2)
            for i in range(n):
                if idx[0, 0, i] not in tour:
                    tour.append(idx[0, 0, i].item())
                    break
            y = torch.tensor(tour).to(DEVICE).unsqueeze(0)
            out = model(x, y)
    
    tour = tour + [0]  # Complete the cycle
    return tour_length(G, tour)

def gap(G, model=None, model_GA=None, random_seed=42, device='cpu'):
    """
    Compute the gap between the optimal solution on graph G and all the analyzed methods
    """
    # Optimal value (hard-coded in the graph)
    TSP = sum([G[i][j]['weight']*G[i][j]['tour'] for (i, j) in G.edges()]) # Optimal

    # Gaps dictionary
    gaps = {'greedy': 0, 'random': 0, 'transformer_tsp': 0, 'transformer_tsp_acc_grad': 0}
    gaps['greedy'] = 100 * (greedy_algorithm(G) - TSP) / TSP
    gaps['random'] = 100 * (random_tour(G, random_seed) - TSP) / TSP
    
    if model is not None:
        gaps['transformer_tsp'] = 100 * (transformer_tsp(G, model, DEVICE=device) - TSP) / TSP
    else:
        gaps['transformer_tsp'] = float('inf')
        
    if model_GA is not None:
        gaps['transformer_tsp_acc_grad'] = 100 * (transformer_tsp(G, model_GA, DEVICE=device) - TSP) / TSP
    else:
        gaps['transformer_tsp_acc_grad'] = float('inf')
    return gaps

In [9]:
# Cell 5: Training Functions
def train_epoch(model, train_loader, optimizer, criterion, device, accumulation_steps=1):
    model.train()
    total_loss = 0
    optimizer.zero_grad()
    
    for i, (x, y) in enumerate(tqdm(train_loader)):
        x, y = x.to(device), y.to(device)
        
        # Shift y for teacher forcing
        decoder_input = y[:, :-1]
        target = y[:, 1:]
        
        # Forward pass
        output = model(x, decoder_input)
        loss = criterion(output.reshape(-1, output.size(-1)), target.reshape(-1))
        
        # Scale loss for gradient accumulation
        loss = loss / accumulation_steps
        loss.backward()
        
        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            optimizer.zero_grad()
            
        total_loss += loss.item() * accumulation_steps
        
    return total_loss / len(train_loader)

def validate(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    
    with torch.no_grad():
        for x, y in val_loader:
            x, y = x.to(device), y.to(device)
            decoder_input = y[:, :-1]
            target = y[:, 1:]
            
            output = model(x, decoder_input)
            loss = criterion(output.reshape(-1, output.size(-1)), target.reshape(-1))
            total_loss += loss.item()
            
    return total_loss / len(val_loader)

In [10]:
# Cell 6: Training Setup and Execution
# Set device



scaler = GradScaler(device='cuda')
scaler_accum = GradScaler(device='cuda')


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

try:
    # Create dataloaders
    train_loader, val_loader, test_loader = create_dataloaders(
        '/kaggle/input/data-t/data/train_20_DLL_ass4.pkl',
        '/kaggle/input/data-t/data/valid_20_DLL_ass4.pkl',
        '/kaggle/input/data-t/data/test_20_DLL_ass4.pkl',
        batch_size=32
    )
    
    # Initialize model and training components
    model = TSPTransformer().to(device)
    optimizer = optim.AdamW(model.parameters(), lr=0.0001,weight_decay=1e-5)
    criterion = nn.CrossEntropyLoss()

    # Training parameters
    n_epochs = 10  # Adjust based on your time budget
    accumulation_steps = 4  # For gradient accumulation training

    # Learning rate schedulers
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)


    # Gradient scaler for mixed precision training
    scaler = GradScaler()
    scaler_accum = GradScaler()

    # Enable cuDNN benchmarking for potentially faster training
    torch.backends.cudnn.benchmark = True


    

    # Training history
    train_losses = []
    val_losses = []

   
except Exception as e:
    print(f"An error occurred during training setup/execution: {str(e)}")
    raise

Using device: cuda


/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:60: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [11]:
#Train model definition



def train_model(model, train_loader, val_loader, criterion, optimizer, device, n_epochs, accumulation_steps=1):
    model.to(device)
    train_losses = []
    val_losses = []

    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()  # Reset gradients at the start of each epoch

        for i, (x, y) in enumerate(tqdm(train_loader, desc=f"Epoch {epoch+1}/{n_epochs}")):
            x, y = x.to(device), y.to(device)
            output = model(x, y[:, :-1])  # Exclude last element of y for input
            loss = criterion(output.view(-1, output.size(-1)), y[:, 1:].contiguous().view(-1))
            
            # Normalize loss to account for batch accumulation
            loss = loss / accumulation_steps
            loss.backward()
            
            if (i + 1) % accumulation_steps == 0 or (i + 1) == len(train_loader):
                optimizer.step()
                optimizer.zero_grad()
            
            total_loss += loss.item() * accumulation_steps

        train_loss = total_loss / len(train_loader)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                output = model(x, y[:, :-1])
                loss = criterion(output.view(-1, output.size(-1)), y[:, 1:].contiguous().view(-1))
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        val_losses.append(val_loss)

        print(f'Epoch {epoch+1}/{n_epochs}')
        print(f'Training Loss: {train_loss:.4f}')
        print(f'Validation Loss: {val_loss:.4f}')
        print('-' * 40)

    return model, train_losses, val_losses

In [ ]:
# Standard Training 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = TSPTransformer().to(device)
model_accum = TSPTransformer().to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.0001,weight_decay=1e-5)
optimizer_accum = optim.AdamW(model_accum.parameters(), lr=0.0001,weight_decay=1e-5)
criterion = nn.CrossEntropyLoss()

# Assuming you have already created your DataLoaders


n_epochs = 10
accumulation_steps = 4

# Learning rate schedulers
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=True)


    # Gradient scaler for mixed precision training
scaler = GradScaler()
scaler_accum = GradScaler()

    # Enable cuDNN benchmarking for potentially faster training
torch.backends.cudnn.benchmark = True







# Standard training
print("Starting standard training...")
model, train_losses, val_losses = train_model(
    model, train_loader, val_loader, criterion, optimizer, device, n_epochs
)


Starting standard training...


Epoch 1/10: 100%|██████████| 1563/1563 [00:59<00:00, 26.11it/s]


Epoch 1/10
Training Loss: 2.1816
Validation Loss: 2.0325
----------------------------------------


Epoch 2/10: 100%|██████████| 1563/1563 [00:59<00:00, 26.25it/s]


Epoch 2/10
Training Loss: 2.0551
Validation Loss: 2.0119
----------------------------------------


Epoch 3/10:   1%|          | 18/1563 [00:00<00:57, 26.81it/s]

In [ ]:

# Training with gradient accumulation
print("Starting training with gradient accumulation...")
model_accum, train_losses_accum, val_losses_accum = train_model(
    model_accum, train_loader, val_loader, criterion, optimizer_accum, device, n_epochs, accumulation_steps
)

# Save models and training history
torch.save(model.state_dict(), 'model_standard.pth')
torch.save(model_accum.state_dict(), 'model_accum.pth')
torch.save({
    'train_losses': train_losses,
    'val_losses': val_losses,
    'train_losses_accum': train_losses_accum,
    'val_losses_accum': val_losses_accum
}, 'training_history.pth')

# Evaluation and plotting
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

def plot_losses(train_losses, val_losses, train_losses_accum, val_losses_accum):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 6))
    
    # Standard training plot
    ax1.plot(train_losses, label='Training Loss')
    ax1.plot(val_losses, label='Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.set_title('Standard Training Losses')
    ax1.legend()
    ax1.grid(True)
    
    # Gradient accumulation training plot
    ax2.plot(train_losses_accum, label='Training Loss')
    ax2.plot(val_losses_accum, label='Validation Loss')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Loss')
    ax2.set_title('Gradient Accumulation Training Losses')
    ax2.legend()
    ax2.grid(True)
    
    plt.tight_layout()
    plt.show()


    # Plot training history
plot_losses(train_losses, val_losses, train_losses_accum, val_losses_accum)


In [ ]:
#Testing

def evaluate_all_methods(test_dataset, model_standard, model_accum, device):
    all_gaps = []
    for data in tqdm(test_dataset.data, desc="Evaluating instances"):
        # If G is part of a tuple, unpack it
        if isinstance(data, tuple):
            G = data[0]  # Assuming the graph is the first element
        else:
            G = data  # Otherwise, use data as-is
        
        gaps = gap(G, model_standard, model_accum, device=device)
        all_gaps.append(gaps)
    
    # Organize results by method
    method_gaps = {
        'random': [g['random'] for g in all_gaps],
        'greedy': [g['greedy'] for g in all_gaps],
        'transformer_tsp': [g['transformer_tsp'] for g in all_gaps],
        'transformer_tsp_acc_grad': [g['transformer_tsp_acc_grad'] for g in all_gaps]
    }
    
    return method_gaps

def plot_gap_comparison(method_gaps):
    plt.figure(figsize=(10, 6))
    plt.boxplot(method_gaps.values(), labels=method_gaps.keys())
    plt.ylabel('Optimality Gap (%)')
    plt.title('Performance Comparison of Different Methods')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.show()

# Plot training history


try:
    # Evaluate all methods on test set
    method_gaps = evaluate_all_methods(test_loader.dataset, model, model_accum, device)
    plot_gap_comparison(method_gaps)

    # Print summary statistics
    print("\nGap Statistics:")
    for method, gaps in method_gaps.items():
        print(f"\n{method}:")
        print(f"Mean gap: {np.mean(gaps):.2f}%")
        print(f"Median gap: {np.median(gaps):.2f}%")
        print(f"Std gap: {np.std(gaps):.2f}%")
except Exception as e:
    print(f"An error occurred during evaluation: {str(e)}")
    raise
